# Algorithms Project: The Big Heist

![bank robbers](./assets/bank-robbers.jpg)
<p style="text-align:right;line-height:0;"><a href='https://www.freepik.com/vectors/background'>Bank robbers vector created by vectorpocket - www.freepik.com</a></p>

## Description

Given list of bank locations, how much money each one holds and the time it would take to rob each one, you need to apply your hard-won algorithm knowledge to make as much profit as possible! _I bet you didn't think that data science could be this profitable!_

You will design and write a solution to an NP-Had algorithm to this problem

You will need to apply all the knowledge learned up to now to get the best possible result at this task given a large list of banks and 3 minutes to run your algorithm.

The list of banks is in `data/bank_data.csv`. This is what the data looks like.

```csv
id, x_coordinate, y_coordinate, money, time (hr)
0, 11.4, 3.3, 5000, 0.6
1, 6.9, 7.1, 15000, 0.3
2, 1.4, 13.2, 900, 1.1
```

You have **24 hours** to make as much money as possible and then escape.

## Rules

- Your run can start anywhere on the map but it has to end at the **helicopter escape zone**: coordinates (0,0)
    - If you try to rob too many banks and can't get to the helicopter in 24 hours, you get caught and go to jail.

- Your solution is a list or array of integers (eg. `[580, 433, 24, 998]`) where the numbers are the IDs of each banks. The ID of each bank is their index (their row index).

- You travel between banks at 30 km/h. You have to travel from one bank to the next!
    - Remember the formula to calculate the distance between two points.
    - The coordinates are in kilometers.
        - So (1, 1) and (1, 2) are one kilometer apart. 
        - This would take 1 / 30 hour = 2 minutes to travel

- Your solution should be an approximative/heuristic algorithm
    - This problem is NP-Hard, you won't find a fast enough algorithm that has the perfect solution
    - It doesn't need to find the best solution
    - Find the best solution you can!

- Your solution has to run on a common laptop in under 3 minutes for the 10,000 cities dataset
    - You can use everything you want to achieve this:
        - Use numpy, pandas, functions, algorithms
        - You can use parallelism
        - Use optimied libraries (pandas, numba, scipy, etc.)
    - Test your code on **small subsets** of the data so they run fast
        - Then scale your code up to bigger chunks of the data

- Your input is a pandas dataframe with the bank data. Your output is a list of bank IDs in order that you rob them:

```python
# example
df = pd.read_csv('bank_data.csv')
robber_algorithm(df)

# Output is a list of bank IDs
[OUTPUT] --> [664, 2341, 26, 998, 9583, 24, 1, 444, 6783]
```

---



In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import sys
from sklearn.neighbors import NearestNeighbors

In [3]:
# import the check solution
sys.path.append('./utils/')
from check_solution import check_solution

# Read the file
df = pd.read_csv('./data/bank_data.csv')

In [8]:
# function to calculate the distance between two points
def distance(x1, y1, x2, y2):
    return math.hypot(x2 - x1, y2 - y1)

# function to calculate the value factor of a bank: money/(rob time + travel time), return both the time spent at this back (including travel time) and the value_factor of this back
def value_factor(money, time, distance):
    ttime = time + distance/30
    value = money/ttime
    return (ttime, value)

In [9]:
def robber_algorithm(df):
    # Create a list of all banks' coordinates (x,y)
    points = list(zip(df['x_coordinate'],df['y_coordinate']))
    # Create a knn model to fit all the points
    knn = NearestNeighbors(n_neighbors = 500).fit(points)

    prev = (0,0) # start from the helicopter zone (0,0) and go backwards
    path = [] # Create an empty list for the travel path
    time_left = 24 # The time limit

    while True:
        values = [] # An empty list to store the values
        time_spent = [] # An empty list to store the time needed to rob this bank including travel time
        # The k closest neighbors for the current point
        nbrs = knn.kneighbors([prev], return_distance = False)[0]
        for row in nbrs:
            if row not in path:
                # calculate the distance between this point and its closest neighbor
                x1, y1 = points[row]
                x2, y2 = prev
                dist = distance(x1, y1, x2, y2)
                # calculate the value_factor of each of the neighbors
                ttime, value = value_factor(df.iloc[row]['money'], df.iloc[row]['time (hr)'], dist)
                values.append(value)
                time_spent.append(ttime)
            else:
                values.append(0)
                time_spent.append(0)
        
        # pick the bank with the highest value factor
        ind = values.index(max(values))
        step=nbrs[ind]
        # if there is still time left to rob this bank, add it to the path, otherwise, stop here
        if time_left - time_spent[ind] >=0:
            path.append(step)
            time_left -= time_spent[ind]
            prev = points[step]
        else:
            break
    
    my_path = path[::-1]
    return my_path


In [10]:
my_path = robber_algorithm(df)

In [11]:
check_solution(my_path, df, speed=30.)

Time Remaining: 0.23430405896293943


14341400.0